# Advanced Classification Predict

© Explore Data Science Academy

## Honour Code

I **CHUENE KOMANE**, confirm - by submitting this document - that the solutions in this notebook are a result of my own work and that I abide by the EDSA honour code (https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).

Non-compliance with the honour code constitutes a material breach of contract.

### Import Libraries and Read In the Data

Do not modify or remove any of the code in these cells.

In [1]:
from IPython.display import Image
from IPython.display import IFrame
from IPython import display
import pandas as pd
import numpy as np

import nltk
import sklearn
import imblearn
import csv
%matplotlib inline

from nltk import TreebankWordTokenizer, SnowballStemmer
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import  word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from imblearn.metrics import classification_report_imbalanced

from sklearn.preprocessing import LabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import VarianceThreshold

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

import string
import urllib
import math
import re

nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
train  = pd.read_csv('train.csv')
test = pd.read_csv('test.csv') # no labels

train.head()

,sentiment,message,tweetid
0,1,PolySciMajor EPA chief doesn't think carbon di...,625221
1,1,It's not like we lack evidence of anthropogeni...,126103
2,2,RT @RawStory: Researchers say we have three ye...,698562
3,1,#TodayinMaker# WIRED : 2016 was a pivotal year...,573736
4,1,"RT @SoyNovioDeTodas: It's 2016, and a racist, ...",466954


In [3]:
test.head()

,message,tweetid
0,Europe will now be looking to China to make su...,169760
1,Combine this with the polling of staffers re c...,35326
2,"The scary, unimpeachable evidence that climate...",224985
3,@Karoli @morgfair @OsborneInk @dailykos \nPuti...,476263
4,RT @FakeWillMoore: 'Female orgasms cause globa...,872928


In [4]:
#train.set_index('tweetid', inplace=True)
#test.set_index('tweetid', inplace=True)
train.head()


,sentiment,message,tweetid
0,1,PolySciMajor EPA chief doesn't think carbon di...,625221
1,1,It's not like we lack evidence of anthropogeni...,126103
2,2,RT @RawStory: Researchers say we have three ye...,698562
3,1,#TodayinMaker# WIRED : 2016 was a pivotal year...,573736
4,1,"RT @SoyNovioDeTodas: It's 2016, and a racist, ...",466954


In [5]:
test.head()

,message,tweetid
0,Europe will now be looking to China to make su...,169760
1,Combine this with the polling of staffers re c...,35326
2,"The scary, unimpeachable evidence that climate...",224985
3,@Karoli @morgfair @OsborneInk @dailykos \nPuti...,476263
4,RT @FakeWillMoore: 'Female orgasms cause globa...,872928


## Overview

Refer to this diagram to guide you while you build your model. Some of the steps will be fleshed out in this template.

![Overview](process_overview_final.jpg)

## Basic preprocessing

Here is a template you may use for your initial base model. 

### Removing URL's
Write a function that removes URL's from a single tweet. 

**Function input:**
- A single string object (tweet) 

**Function output:**
- The tweet with URL's removed as a single object

In [6]:
# define function that removes URL from single tweet
def remove_urls(text):
    no_url = re.sub(r"http\S+", "", text)
    return no_url

**Original object:**

In [7]:
tweet = train['message'][2]
tweet

'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late https://t.co/WdT0KdUr2f https://t.co/Z0ANPT…'

In [8]:
remove_urls(tweet)

'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late  '

**Expected output:**
```python
remove_urls(tweet)

>>'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late  '
```

### Convert to lowercase

Write a function that converts a single tweet to lowercase.

**Function input:**
- A single string object (tweet) 

**Function output:**
- The tweet in lowercase as a single object

In [9]:
def to_lower(text):
    lowercase=text.lower()
    return lowercase

**Original object:**

In [10]:
tweet

'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late https://t.co/WdT0KdUr2f https://t.co/Z0ANPT…'

In [11]:
to_lower(tweet)

'rt @rawstory: researchers say we have three years to act on climate change before it’s too late https://t.co/wdt0kdur2f https://t.co/z0anpt…'

**Expected output:**

```python 
to_lower(tweet)
>> 'rt @rawstory: researchers say we have three years to act on climate change before it’s too late https://t.co/wdt0kdur2f https://t.co/z0anpt…'
```

### Stemming or lemmatising

Write a function that derives the root words for each of the words in a tweet. You may use stemming _or_ lemmatising here.

**Function input:**
- A single string object (tweet) 

**Function output:**
- The tweet in root words as a _single_ object

In [12]:
def get_roots(text):
    lemmatizer = WordNetLemmatizer()
    return "".join([lemmatizer.lemmatize(word) for word in text])

**Original object:**

In [13]:
tweet

'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late https://t.co/WdT0KdUr2f https://t.co/Z0ANPT…'

In [14]:
get_roots(tweet)

'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late https://t.co/WdT0KdUr2f https://t.co/Z0ANPT…'

**Expected output:**
```python
get_roots(tweet)
```
Using `SnowballStemmer()`
```python
>> 'rt @rawstory: research say we have three year to act on climat chang befor it too late https://t.co/wdt0kdur2f https://t.co/z0anpt…'
```

Using `WordNetLemmatizer()`
```python
>> 'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late https://t.co/WdT0KdUr2f https://t.co/Z0ANPT…'
```

### Other preprocessing techniques 

Come back here once you have submitted your base model to add any other preprocessing functions yuo wish to add

In [15]:
# YOUR CODE HERE


In [16]:
def remove_punctuation(post):
    return ''.join([l for l in post if l not in string.punctuation])

In [17]:
remove_punctuation(tweet)

'RT RawStory Researchers say we have three years to act on climate change before it’s too late httpstcoWdT0KdUr2f httpstcoZ0ANPT…'

### Tokenising 

Write a function that tokenises a single tweet into a list of tokens.

**Function input:**
- A single string object (tweet) 

**Function output:**
- A list of tokens (objects).

In [18]:
def get_tokens(text):
    tokeniser = TreebankWordTokenizer()
    return tokeniser.tokenize(text)

**Original object:**

In [19]:
tweet

'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late https://t.co/WdT0KdUr2f https://t.co/Z0ANPT…'

In [20]:
get_tokens(tweet)

['RT',
 '@',
 'RawStory',
 ':',
 'Researchers',
 'say',
 'we',
 'have',
 'three',
 'years',
 'to',
 'act',
 'on',
 'climate',
 'change',
 'before',
 'it’s',
 'too',
 'late',
 'https',
 ':',
 '//t.co/WdT0KdUr2f',
 'https',
 ':',
 '//t.co/Z0ANPT…']

**Expected output:**
```python
get_tokens(tweet)

>> ['RT',
 '@',
 'RawStory',
 ':',
 'Researchers',
 'say',
 'we',
 'have',
 'three',
 'years',
 'to',
 'act',
 'on',
 'climate',
 'change',
 'before',
 'it’s',
 'too',
 'late',
 'https',
 ':',
 '//t.co/WdT0KdUr2f',
 'https',
 ':',
 '//t.co/Z0ANPT…']
```

Besides the preprocessing methods we've already discussed, can you think of how our data can be improved at this stage?

### Vectorising 

Use the code from the webinar to define your vecotiser. For more details on this function have a look at the `sklearn`
<a href="https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html">package documentation on the function</a>. Recall that this function, unlike the others before takes an _**entire column**_ as an argument.

### Putting all the functions together 

Test your cleaning functions on a single tweet. Perform the cleaning procedure on this tweet from start to finish. Make sure the output looks as you would expect. Store the cleaned tweet in `tweet_final`. Don't worry about vectorising for now.

**_Include any other cleaning procedures you add after your base model_**

**Original object:**

In [21]:
tweet

'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late https://t.co/WdT0KdUr2f https://t.co/Z0ANPT…'

In [22]:
# clean tweet from start to finish
tweet1 = remove_urls(tweet)
tweet2 = to_lower(tweet1)
tweet3=get_roots(tweet2)
tweet4=remove_punctuation(tweet3)
tweet_final=get_tokens(tweet4)
#.
#.

# cleaned tweet
tweet_final

['rt',
 'rawstory',
 'researchers',
 'say',
 'we',
 'have',
 'three',
 'years',
 'to',
 'act',
 'on',
 'climate',
 'change',
 'before',
 'it’s',
 'too',
 'late']

**Expected output:**
```python
tweet_final
>>['rt',
 '@',
 'rawstory',
 ':',
 'research',
 'say',
 'we',
 'have',
 'three',
 'year',
 'to',
 'act',
 'on',
 'climat',
 'chang',
 'befor',
 'it',
 'too',
 'late']
```

### Preprocessing and cleaning entire column of tweets

Now that we know how to clean a single tweet, we can use `apply()` to clean the entire column in the dataframe. This is what the column looks like to begin with.

In [23]:
train['message'].head()

0    PolySciMajor EPA chief doesn't think carbon di...
1    It's not like we lack evidence of anthropogeni...
2    RT @RawStory: Researchers say we have three ye...
3    #TodayinMaker# WIRED : 2016 was a pivotal year...
4    RT @SoyNovioDeTodas: It's 2016, and a racist, ...
Name: message, dtype: object

Write code that applies your cleaning functions to the entire `message` column of the dataframe using `.apply()`. Do not vectorise just yet. Store this in a new column, `train['message_clean']`. 

In [24]:
# applying cleaning functions to entire column of dataframe (NO VECTORISING)

# YOUR CODE HERE - use apply with all your cleaning functions 

#train['message_clean'] = # YOUR CLEANED DATA COLUMN (WITHOUT VECTORISING)
train['message_clean']=train['message'].apply(remove_urls).apply(to_lower).apply(get_roots).apply(get_tokens)
train['message_clean'].head()

0    [polyscimajor, epa, chief, does, n't, think, c...
1    [it, 's, not, like, we, lack, evidence, of, an...
2    [rt, @, rawstory, :, researchers, say, we, hav...
3    [#, todayinmaker, #, wired, :, 2016, was, a, p...
4    [rt, @, soynoviodetodas, :, it, 's, 2016, ,, a...
Name: message_clean, dtype: object

**Expected output:**

```python
train['message_clean'].head()

>>  0    [polyscimajor, epa, chief, does, n't, think, c...
    1    [it, not, like, we, lack, evid, of, anthropoge...
    2    [rt, @, rawstory, :, research, say, we, have, ...
    3    [#, todayinmaker, #, wire, :, 2016, was, a, pi...
    4    [rt, @, soynoviodetodas, :, it, 2016, ,, and, ...
Name: message_clean, dtype: object
```

In [25]:
train.head()

,sentiment,message,tweetid,message_clean
0,1,PolySciMajor EPA chief doesn't think carbon di...,625221,"[polyscimajor, epa, chief, does, n't, think, c..."
1,1,It's not like we lack evidence of anthropogeni...,126103,"[it, 's, not, like, we, lack, evidence, of, an..."
2,2,RT @RawStory: Researchers say we have three ye...,698562,"[rt, @, rawstory, :, researchers, say, we, hav..."
3,1,#TodayinMaker# WIRED : 2016 was a pivotal year...,573736,"[#, todayinmaker, #, wired, :, 2016, was, a, p..."
4,1,"RT @SoyNovioDeTodas: It's 2016, and a racist, ...",466954,"[rt, @, soynoviodetodas, :, it, 's, 2016, ,, a..."


### Vectorising 

Vectorise your cleaned data using the vectoriser you defined earlier. Don't forget to fit the vectoriser to the data you just cleaned. Store your vectorised data in `train_vec`.

In [26]:
# vectorise your cleaned data
from sklearn.feature_extraction.text import TfidfVectorizer
tt = TfidfVectorizer(preprocessor=list, tokenizer=list, ngram_range=(1,2),min_df=2, strip_accents='ascii',smooth_idf=False)
tt.fit(train['message_clean'])
train_vec=tt.transform(train['message_clean'])
# YOUR CODE HERE 

C:\Users\User\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [27]:
train_vec.shape

(15819, 37610)

In [28]:
train_vec.toarray()

array([[0.10692521, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

## Fitting a simple model on training data and evaluate its performance

Fit a model on your cleaned data. Use `train_vec` as your features. Note that you need to convert `train_vec` to an array. 

You are familiar with this process, so there is less guidance here. You may also refer to train notebooks and the webinar pdf as guidance.

Your basic model should be a logistic regression, however if you try different models you will also perform the following procedure.

1. Split the training data into features and labels.
2. Split the training data further into training and validation data.
3. Fit the model on the training subset. 
4. Predict on the validation subset.
5. Calculate the performance metrics on the validation predictions.
6. Select a model based on validation performance (when you have more than one model).
8. Clean the test data.
9. Predict on the cleaned test data.
7. Write a csv that matches `sample_submission.csv`.
8. Submit to Kaggle.

In [29]:
# YOUR CODE HERE



In [30]:
y=train['sentiment']
y.head()

0    1
1    1
2    2
3    1
4    1
Name: sentiment, dtype: int64

In [31]:
X=train_vec
X.shape

(15819, 37610)

In [32]:
X

<15819x37610 sparse matrix of type '<class 'numpy.float64'>'
	with 505524 stored elements in Compressed Sparse Row format>

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=27)

In [34]:
lr=LogisticRegression(multi_class='ovr')
lr.fit(X_train,y_train)


LogisticRegression(multi_class='ovr')

In [35]:
y_pred=lr.predict(X_test)

In [36]:
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

         -1       0.86      0.17      1.00      0.29      0.42      0.16       246
          0       0.65      0.28      0.97      0.40      0.53      0.26       455
          1       0.72      0.93      0.56      0.81      0.73      0.55      1730
          2       0.81      0.75      0.95      0.78      0.84      0.70       733

avg / total       0.74      0.74      0.75      0.71      0.70      0.51      3164



In [37]:
f1=f1_score(y_test, y_pred, average="macro")
print(f1)

0.5702799456640667


CLEANING THE TEST DATA SET

In [38]:
test['message_clean']=test['message'].apply(remove_urls).apply(to_lower).apply(get_roots).apply(get_tokens)
test['message_clean'].head()

0    [europe, will, now, be, looking, to, china, to...
1    [combine, this, with, the, polling, of, staffe...
2    [the, scary, ,, unimpeachable, evidence, that,...
3    [@, karoli, @, morgfair, @, osborneink, @, dai...
4    [rt, @, fakewillmoore, :, 'female, orgasms, ca...
Name: message_clean, dtype: object

In [39]:
test_vec=tt.transform(test['message_clean'])


In [40]:
test_vec.shape

(10546, 37610)

In [41]:
y_test_pred=lr.predict(test_vec)

## Suggestions on how to improve the model - Extras 

Here are some ideas on how you can improve your model. Try some of them out once you have submitted your base model to Kaggle. Only use the methods you find improve your model performance.

 - Additional preprocessing 
     - Use some of the methods in the notebooks to balance the data
     - Replace links with the text `Link` to show the machine that there is a link present without the extraneous detail of the link text
     - Remove punctuation
     - Correct spelling
     - Bag of words vs. N-grams - which is better?
     - Try `TweetTokenizer()` - how does this differ from the tokenisers you are familiar with?
 - Consider which other models could do a better job (refer to course resources)
 - Consider some further feature engineering:
     - What other attributes of the tweets may be useful?
     - Should we perform feature selection given their size and sparseness of the vectorised tweets?
 - If you've mastered the above, try implementing hyperparameter tuning methods.
 - Compare all the models you try and remember to submit **the best one** based on test performance (f1-score using macro averaging). 
 
You may use the space below to try out these "Extras".

## How/why is the model bad?

Do you have any hypotheses as to how you can improve the model further? Use this space to investigate with mindful EDA if necessary.

In [42]:
# EDA
# YOUR CODE HERE 
#balancing the classes didn't improve performance?
#removing punctuation and stopwords didn't improve performance?





Be sure to check whether each additional technique you apply actually improves your model performance.

## Submit your best model predictions 

Don't forget to submit your the predictions on the best model to Kaggle.

In [43]:
# write csv for best predictions

In [44]:
pred_submit=pd.DataFrame(y_test_pred)
pred_submit.head()

,0
0,1
1,1
2,1
3,1
4,0


In [45]:

# tweetids
ids_submit = test[['tweetid']]
# create dataframeand name columns as needed 
submit_all = pd.concat([ids_submit.reset_index(drop=True), pred_submit], axis=1).rename(columns = {0:'sentiment'})
submit_all.head()


,tweetid,sentiment
0,169760,1
1,35326,1
2,224985,1
3,476263,1
4,872928,0


In [46]:
submit_all.to_csv(r'final_submission.csv', index = False) 